1. Fetch dataand load dataset
2. split train-70% val-30%
3. Train loader, val loader
4. Prepare model
5. Loss function
6. optimizer
7. Plot graphs w.r.t loss vs train accuracy and val accuracy

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

import torch
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [47]:
df = pd.read_csv(Path.joinpath(Path.cwd().parent,'dataset','5_model_df.csv'))
df.drop(columns=['runs_off_bat', 'extras','player_dismissed'],inplace=True)
df

,venue,innings,batting_team,bowling_team,bat1,bat2,bat3,bat4,bat5,bat6,...,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6,Total_score
0,15,1,2,5,220,36,205.0,0.0,0.0,0.0,...,0.0,0.0,0.0,171,275,16.0,0.0,0.0,0.0,61
1,15,2,5,2,186,264,257.0,97.0,51.0,155.0,...,0.0,0.0,0.0,18,109,8.0,0.0,0.0,0.0,26
2,23,1,0,3,174,148,141.0,0.0,0.0,0.0,...,0.0,0.0,0.0,46,268,105.0,0.0,0.0,0.0,53
3,23,2,3,0,107,105,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,126,200,0.0,0.0,0.0,0.0,63
4,10,1,6,1,252,270,235.0,130.0,0.0,0.0,...,0.0,0.0,0.0,96,43,189.0,0.0,0.0,0.0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1,2,4,0,183,196,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,82,225,171.0,0.0,0.0,0.0,58
1455,1,1,6,7,94,269,240.0,0.0,0.0,0.0,...,0.0,0.0,0.0,44,297,254.0,154.0,0.0,0.0,42
1456,1,2,7,6,146,100,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,167,208,58.0,49.0,0.0,0.0,57
1457,18,1,3,1,173,136,48.0,63.0,0.0,0.0,...,0.0,0.0,0.0,109,152,109.0,40.0,0.0,0.0,39


In [48]:
input_params = ['venue', 'innings', 'batting_team', 'bowling_team', 'bat1', 'bat2', 'bat3', 'bat4', 'bat5', 'bat6',
                    'bat7', 'bat8', 'bat9', 'bat10', 'bow1', 'bow2', 'bow3', 'bow4', 'bow5', 'bow6']

output_param = ['Total_score']

# Normalize DataFrame

In [49]:
mms = MinMaxScaler()
df_norm = pd.DataFrame(mms.fit_transform(df),columns=df.columns)
df_norm

,venue,innings,batting_team,bowling_team,bat1,bat2,bat3,bat4,bat5,bat6,...,bat8,bat9,bat10,bow1,bow2,bow3,bow4,bow5,bow6,Total_score
0,0.441176,0.0,0.285714,0.714286,0.802198,0.129630,0.745455,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.513595,0.827273,0.048193,0.000000,0.0,0.0,0.572816
1,0.441176,1.0,0.714286,0.285714,0.677656,0.974074,0.934545,0.354015,0.186131,0.574074,...,0.0,0.0,0.0,0.051360,0.324242,0.024096,0.000000,0.0,0.0,0.233010
2,0.676471,0.0,0.000000,0.428571,0.633700,0.544444,0.512727,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.135952,0.806061,0.316265,0.000000,0.0,0.0,0.495146
3,0.676471,1.0,0.428571,0.000000,0.388278,0.385185,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.377644,0.600000,0.000000,0.000000,0.0,0.0,0.592233
4,0.294118,0.0,0.857143,0.142857,0.919414,0.996296,0.854545,0.474453,0.000000,0.000000,...,0.0,0.0,0.0,0.287009,0.124242,0.569277,0.000000,0.0,0.0,0.368932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,0.029412,1.0,0.571429,0.000000,0.666667,0.722222,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.244713,0.675758,0.515060,0.000000,0.0,0.0,0.543689
1455,0.029412,0.0,0.857143,1.000000,0.340659,0.992593,0.872727,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.129909,0.893939,0.765060,0.463855,0.0,0.0,0.388350
1456,0.029412,1.0,1.000000,0.857143,0.531136,0.366667,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.501511,0.624242,0.174699,0.147590,0.0,0.0,0.533981
1457,0.529412,0.0,0.428571,0.142857,0.630037,0.500000,0.174545,0.229927,0.000000,0.000000,...,0.0,0.0,0.0,0.326284,0.454545,0.328313,0.120482,0.0,0.0,0.359223


# Split Dataset 

In [59]:
train_ds,val_ds = train_test_split(df_norm,test_size=0.2,shuffle=True)
print(train_ds.shape,val_ds.shape)

(1167, 21) (292, 21)


In [57]:
tmp = torch.Tensor(train_ds.values.astype(np.float32))
tmp

tensor([[ 10.,   2.,   7.,   1.,  21., 210.,  55., 168.,   0.,   0.,   0.,   0.,
           0.,   0., 204., 261., 114., 236., 127.,   0.,  44.],
        [ 34.,   1.,   4.,   6., 196., 183.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0., 108.,  89.,  93., 125.,   0.,   0.,  57.],
        [  8.,   2.,   4.,   1., 196., 183., 218.,   0.,   0.,   0.,   0.,   0.,
           0.,   0., 184., 109.,  10., 152.,  27.,   0.,  61.],
        [ 21.,   1.,   1.,   3.,  76., 258., 254.,  11.,   0.,   0.,   0.,   0.,
           0.,   0.,  46.,  87., 180., 268.,   0.,   0.,  38.],
        [ 20.,   1.,   4.,   0., 238., 234., 210.,   0.,   0.,   0.,   0.,   0.,
           0.,   0., 200., 306.,   2.,   0.,   0.,   0.,  40.]])

In [58]:
tmp[:,-1]

tensor([44., 57., 61., 38., 40.])

In [62]:
x_train = torch.tensor(train_ds[input_params].values.astype(np.float32))

y_train = torch.tensor(train_ds[output_param].values.astype(np.float32))

print(x_train.shape,y_train.shape)

torch.Size([1167, 20]) torch.Size([1167, 1])


In [63]:
x_val = torch.tensor(val_ds[input_params].values.astype(np.float32))

y_val = torch.tensor(val_ds[output_param].values.astype(np.float32))

print(x_val.shape,y_val.shape)

torch.Size([292, 20]) torch.Size([292, 1])


In [64]:
x_train_dl = torch.utils.data.DataLoader(x_train,batch_size=10,shuffle=True)
y_train_dl = torch.utils.data.DataLoader(y_train,batch_size=10,shuffle=True)

x_val_dl = torch.utils.data.DataLoader(x_val,batch_size=10,shuffle=True)
y_val_dl = torch.utils.data.DataLoader(y_val,batch_size=10,shuffle=True)



# Define model

In [67]:
model = torch.nn.Linear(20, 1)
print(model.weight.shape)
print(model.bias.shape)

torch.Size([1, 20])
torch.Size([1])


In [74]:
dl = iter(x_train_dl)
for j in dl.next():
    print(model(j),)
    #break

tensor([-0.0149], grad_fn=<AddBackward0>)
tensor([0.1831], grad_fn=<AddBackward0>)
tensor([0.3368], grad_fn=<AddBackward0>)
tensor([0.0714], grad_fn=<AddBackward0>)
tensor([0.1234], grad_fn=<AddBackward0>)
tensor([-0.0606], grad_fn=<AddBackward0>)
tensor([0.0422], grad_fn=<AddBackward0>)
tensor([0.1370], grad_fn=<AddBackward0>)
tensor([0.0621], grad_fn=<AddBackward0>)
tensor([0.0449], grad_fn=<AddBackward0>)
